In [ ]:
gdbs = list()
for dirpath, dirs, files in os.walk(r'scenario'):
    for d in dirs:
        dir_name = os.path.join(base_path, dirpath, d)
        if os.path.isdir(dir_name) and dir_name.endswith('.gdb'):
            if not d == 'scenario_template.gdb':
                gdbs.append(os.path.join(dirpath, d))
for mode in ['auto', 'bike', 'transit']:
    scenario_folder = os.path.join('scenario', mode)
    for file in os.listdir(scenario_folder):
        d = os.path.join(base_path, scenario_folder, file)
        if os.path.isdir(d) and dir_name.endswith('.gdb'):
            if arcpy.Exists(os.path.join(d, 'scores')) == False:
                scoring.score(gdb, mode = mode)

In [3]:
for mode in ['auto', 'bike', 'transit']:
    scenario_folder = os.path.join('scenario', mode)
    for file in os.listdir(scenario_folder):
        d = os.path.join(base_path, scenario_folder, file)
        if os.path.isdir(d) and dir_name.endswith('.gdb'):
            if arcpy.Exists(os.path.join(d, 'scores')) == False:
                print('Scoring {0} for {1}'.format(gdb, mode))

NameError: name 'os' is not defined

In [ ]:
before = pd.read_csv("bus_baseline.csv")
after = pd.read_csv("mod.csv")
compare_out_file = mode + "_mod_compare.csv"

In [ ]:
df = pd.merge(before, 
              after, 
              on=['Origin_TAZID', 'Destination_TAZID'], 
              how="inner", 
              suffixes=("_before", "_after")
)

In [ ]:
df['diff_hh'] = df['weighted_hh_after'] - df['weighted_hh_before']
df['diff_jobs'] = df['weighted_jobs_after'] - df['weighted_jobs_before']
df['diff_ato'] = df['ato_after'] - df['ato_before']

In [ ]:
#df.groupby('Origin_CO_TAZID_before')['diff_ato'].sum()

In [ ]:
df.head()

In [ ]:
# assert df['diff_ato'].min() >= 0

In [ ]:
summary = df.groupby('Origin_TAZID').agg(
    jobs=pd.NamedAgg(column='diff_jobs', aggfunc=sum),
    hh=pd.NamedAgg(column='diff_hh', aggfunc=sum),
    ato=pd.NamedAgg(column='diff_ato', aggfunc=sum))

In [ ]:
summary['ato'] = summary['jobs'] + summary['hh'] * 1.8

In [ ]:
summary.to_csv(compare_out_file)

In [ ]:
df.agg(
    jobs=pd.NamedAgg(column='diff_jobs', aggfunc=sum),
    hh=pd.NamedAgg(column='diff_hh', aggfunc=sum),
    ato=pd.NamedAgg(column='diff_ato', aggfunc=sum))

## Visualize Output

In [6]:
arcpy.management.MakeFeatureLayer(
    os.path.join(base_path, r"shp\taz_wfrc.gdb\ATO"), 
    "taz_compare"
)

<Result 'taz_compare'>

In [ ]:
arcpy.conversion.TableToTable(
    os.path.join(base_path, compare_out_file), 
    base_gdb , 
    "mod_compare"
)

In [ ]:
arcpy.management.AddJoin(
    "taz_compare", 
    "CO_TAZID", 
    "mod_compare", 
    "Origin_TAZID", 
    "KEEP_ALL"
)

Now, manually apply symbology to taz_compare layer (I experimented with creating a layer style and applying programatically), but found it easier just to manually update)
